In [1]:
# Dependencies
import pandas as pd
from pathlib import Path
import time
# used for interactive mapping 
import geopandas
import folium

>### <font color=darkblue>Import, Clean, & Narrow Station Data to Electric (ELEC) Stations only </font>
***
__Cleaned some of the data in CSV prior to import by removing unwanted columns__
  
- __Query Parameters:__1. _Passenger vehicles (class 1-2)_
2. _US only stations_
3. _Station is status 'Available'_
4. _Fuel Types (Biodiesel, Electric, Ethanol (E85), Renewable Diesel)_
5. _Station is 'public'_ic




In [2]:
# Name of the Cleaned Station CSV file and read it in

US_stations_df = pd.read_csv('Resources/Cleaned_all_US_fuel_stations_1-28-RM.csv', encoding= 'ISO-8859-1')


In [3]:
# review US_stations df
US_stations_df.head(2)

,ï»¿Fuel _Type_Code,Station_Name,Street_Address,City,State,ZIP,Station_Phone,Access_Days_Time,Cards_Accepted,EV_Level1_EVSE_Num,EV_Level2_EVSE_Num,EV_DC_Fast_Count,Geocode_Status,Latitude,Longitude,EV_Connector_Types,Country,Facility_Type,E85_Other_Ethanol_Blends
0,E85,Sioux Valley Co-op - Cenex,220 10th St NW,Watertown,SD,57201,605-886-5829,24 hours daily,A Cash Checks CREDIT D Debit M V Voyager Wrigh...,NaN,NaN,NaN,200-9,44.904113,-97.130798,NaN,US,COOP,"[""E30-E35""]"
1,E85,F&M Co-op Oil,45316 Highway 34,Madison,SD,57042,605-256-4516,"6am-10pm M-Sat, 8am-10pm Sun",A Cash D M V Wright_Exp,NaN,NaN,NaN,200-8,44.007690,-97.146785,NaN,US,COOP,"[""E20-E25"", ""E30-E35""]"


In [4]:
# rename fuel type column, access_days_time, & station charging levels, cards_accepted 

renamed_US_stations_df = US_stations_df.rename(columns={"ï»¿Fuel _Type_Code": "Fuel_Type", "Access_Days_Time": "Business_Hours", "EV_Level1_EVSE_Num": "Count_Level_1","EV_Level2_EVSE_Num": "Count_Level_2", "EV_DC_Fast_Count": "Count_DC_Fast", "Cards_Accepted": "Payment"})
renamed_US_stations_df.head(2)

,Fuel_Type,Station_Name,Street_Address,City,State,ZIP,Station_Phone,Business_Hours,Payment,Count_Level_1,Count_Level_2,Count_DC_Fast,Geocode_Status,Latitude,Longitude,EV_Connector_Types,Country,Facility_Type,E85_Other_Ethanol_Blends
0,E85,Sioux Valley Co-op - Cenex,220 10th St NW,Watertown,SD,57201,605-886-5829,24 hours daily,A Cash Checks CREDIT D Debit M V Voyager Wrigh...,NaN,NaN,NaN,200-9,44.904113,-97.130798,NaN,US,COOP,"[""E30-E35""]"
1,E85,F&M Co-op Oil,45316 Highway 34,Madison,SD,57042,605-256-4516,"6am-10pm M-Sat, 8am-10pm Sun",A Cash D M V Wright_Exp,NaN,NaN,NaN,200-8,44.007690,-97.146785,NaN,US,COOP,"[""E20-E25"", ""E30-E35""]"


In [5]:
# review data types in the df 
renamed_US_stations_df.dtypes

Fuel_Type                    object
Station_Name                 object
Street_Address               object
City                         object
State                        object
ZIP                           int64
Station_Phone                object
Business_Hours               object
Payment                      object
Count_Level_1               float64
Count_Level_2               float64
Count_DC_Fast               float64
Geocode_Status               object
Latitude                    float64
Longitude                   float64
EV_Connector_Types           object
Country                      object
Facility_Type                object
E85_Other_Ethanol_Blends     object
dtype: object

In [6]:
# use value_counts() to count occurrences of 'Fuel_Types' (Biodiesel, Ele, Ethanol (E85), Renewable Diesel)

# Biodiesel ('BD')
count = renamed_US_stations_df['Fuel_Type'].value_counts()['BD']

print(f"The number stations for fuel type Biodiesel: {count}")

# Electic ('ELEC')
count = renamed_US_stations_df['Fuel_Type'].value_counts()['ELEC']

print(f"The number stations for fuel type Electric: {count}")

# Ethanol ('E85')
count = renamed_US_stations_df['Fuel_Type'].value_counts()['E85']

print(f"The number stations for fuel type Ethanol 85: {count}")


# Renewable Diesel ('RD')
count = renamed_US_stations_df['Fuel_Type'].value_counts()['RD']

print(f"The number stations for fuel type Renewable Diesel: {count}")


The number stations for fuel type Biodiesel: 1113
The number stations for fuel type Electric: 9087
The number stations for fuel type Ethanol 85: 3177
The number stations for fuel type Renewable Diesel: 43


In [13]:
#filter the df to only Electric stations 
ELEC_Stations_in_USA = renamed_US_stations_df.loc[(renamed_US_stations_df.Fuel_Type == "ELEC")]

#print ELEC Stations only DF
ELEC_Stations_in_USA.head()

,Fuel_Type,Station_Name,Street_Address,City,State,ZIP,Station_Phone,Business_Hours,Payment,Count_Level_1,Count_Level_2,Count_DC_Fast,Geocode_Status,Latitude,Longitude,EV_Connector_Types,Country,Facility_Type,E85_Other_Ethanol_Blends
5,ELEC,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,213-741-1151,5:30am-9pm; pay lot,NaN,NaN,7.0,NaN,GPS,34.040539,-118.271387,J1772,US,PARKING_GARAGE,NaN
7,ELEC,Galpin Motors,15421 Roscoe Blvd,Sepulveda,CA,91343,800-256-6219,Dealership business hours; customer use only,NaN,NaN,2.0,NaN,200-9,34.221665,-118.468371,J1772,US,CAR_DEALER,NaN
8,ELEC,Galleria at Tyler,1299 Galleria at Tyler,Riverside,CA,92503,951-351-3110,6am-12am daily,NaN,NaN,4.0,NaN,GPS,33.909914,-117.459053,J1772,US,SHOPPING_MALL,NaN
9,ELEC,City of Pasadena - Holly Street Garage,150 E Holly St,Pasadena,CA,91103,626-744-7665,24 hours daily; pay lot,NaN,NaN,16.0,NaN,200-8,34.147620,-118.147111,J1772,US,PARKING_GARAGE,NaN
10,ELEC,City of Pasadena - De Lacey Garage,45 De Lacey Ave,Pasadena,CA,91105,626-744-7665,24 hours daily; pay lot,NaN,NaN,2.0,NaN,GPS,34.145138,-118.152655,J1772,US,PARKING_GARAGE,NaN


In [14]:
#count of rows in the elec stations df, to ensure I got all the electric stations 

len(ELEC_Stations_in_USA)

9087

>### <font color=darkblue>JOE- Create Choropleth of all Electric Stations in the US </font>
***
- station density color scale legend
- state hover info: count of stations with level 1, level 2, DC fast
- should be able to remove state fill color from the choropleth 


In [15]:
# use the "ELEC_Stations_in_USA" df to create this 
#joe to do using geopandas and folium 
#see this link: https://geopandas.org/en/stable/docs/user_guide/interactive_mapping.html




>### <font color=darkblue>RAGHEN- Create layer of makers for only DC fast stations on Choropleth of all Electric Stations in the US </font>
***
- Marker Hover Info: station name, address, phone, facility type, business hours
- should be able to turn markers on and off in the legend


In [16]:
# refresher on the df info
ELEC_Stations_in_USA.head(2)

,Fuel_Type,Station_Name,Street_Address,City,State,ZIP,Station_Phone,Business_Hours,Payment,Count_Level_1,Count_Level_2,Count_DC_Fast,Geocode_Status,Latitude,Longitude,EV_Connector_Types,Country,Facility_Type,E85_Other_Ethanol_Blends
5,ELEC,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,213-741-1151,5:30am-9pm; pay lot,NaN,NaN,7.0,NaN,GPS,34.040539,-118.271387,J1772,US,PARKING_GARAGE,NaN
7,ELEC,Galpin Motors,15421 Roscoe Blvd,Sepulveda,CA,91343,800-256-6219,Dealership business hours; customer use only,NaN,NaN,2.0,NaN,200-9,34.221665,-118.468371,J1772,US,CAR_DEALER,NaN


In [21]:
# Make my df to add markers and hover- narrow df to only have stations with DC fast chargers 

#drop columns- payment, level 1, level 2,country, geo_code etc.)

new_DC_Fast_df= ELEC_Stations_in_USA.drop(['Payment', 'Count_Level_1', 'Count_Level_2', 'E85_Other_Ethanol_Blends', 'Country', 'Geocode_Status', 'EV_Connector_Types' ], axis=1)

new_DC_Fast_df.head()

,Fuel_Type,Station_Name,Street_Address,City,State,ZIP,Station_Phone,Business_Hours,Count_DC_Fast,Latitude,Longitude,Facility_Type
845,ELEC,City of Greenville - Richardson Garage,66 Richardson St,Greenville,SC,29601,864-467-4900,24 hours daily; pay lot,1.0,34.852437,-82.399901,PAY_GARAGE
860,ELEC,City of Santa Monica - Civic Parking Garage,333 Civic Center Dr,Santa Monica,CA,90401,310-458-8516,24 hours daily; pay lot,2.0,34.010528,-118.489800,MUNI_GOV
861,ELEC,Denver International Airport - Canopy Airport ...,8100 Tower Rd,Commerce City,CO,80022,303-574-9800,24 hours daily; pay lot,1.0,39.843886,-104.771138,PARKING_LOT
870,ELEC,Clay Cooley Nissan,4914 S IH-35,Austin,TX,78745,512-444-1400,Dealership business hours,1.0,30.206330,-97.759360,CAR_DEALER
905,ELEC,Town North Nissan,9160A Research Blvd,Austin,TX,78758,512-451-7411,Dealership business hours,1.0,30.374542,-97.731445,CAR_DEALER


In [19]:
len(new_DC_Fast_df)

2085

In [26]:
# Ensure there are no null values for DC Fast- drop null values from DC Fast column

new_DC_Fast_df.dropna(subset=['Count_DC_Fast'], inplace=True)

In [27]:
len(new_DC_Fast_df)

2085

In [49]:
# Converting 'Facility_Type', 'Business_Hours', 'Station_Phone' to string
new_DC_Fast_df[['Facility_Type', 'Business_Hours', 'Station_Phone']] = new_DC_Fast_df[['Facility_Type', 'Business_Hours', 'Station_Phone']].astype(str)
# Displaying the types of data after conversion
print("\nTypes of data after conversion:\n", new_DC_Fast_df.dtypes)


Types of data after conversion:
 Fuel_Type          object
Station_Name       object
Street_Address     object
City               object
State              object
ZIP                 int64
Station_Phone      object
Business_Hours     object
Count_DC_Fast     float64
Latitude          float64
Longitude         float64
Facility_Type      object
dtype: object


In [50]:
#create the base map using a central location around Montana, OpenStreetMap, and zoom of 3

base_map = folium.Map(location=[48, -102], tiles='OpenStreetMap',\
                      zoom_start=4)

base_map

In [51]:
# loop through the DataFrame to read the longitudinal and latitudinal values from the columns df[‘Longitude’] and df[‘Latitude’]

for i, row in new_DC_Fast_df.iterrows():
    lat = new_DC_Fast_df.at[i, 'Latitude']
    lng = new_DC_Fast_df.at[i, 'Longitude']


In [52]:
# We will also create a pop-up to show the station name, facility type, business hours, address, phone within the loop.

#just station name to see if it works
popup = 'Station : ' + str(new_DC_Fast_df.at[i, 'Station_Name'])  


In [58]:
# Withing the loop, pass the latitude and longitude values to the location variable, popup details, and folium icon details

#create map
base_map = folium.Map(location=[48, -102], tiles='OpenStreetMap', zoom_start=4)

#loop to get lat and lng values
for i, row in new_DC_Fast_df.iterrows():
    lat = new_DC_Fast_df.at[i, 'Latitude']
    lng = new_DC_Fast_df.at[i, 'Longitude']
    
    #hover state (station name, address, facility type, phone, hours)
    popup = 'Station : ' + str(new_DC_Fast_df.at[i, 'Station_Name']) + '<br>' + '<br>' +'Address: ' + new_DC_Fast_df.at[i, 'Street_Address'] + '<br>' + '<br>' +'Facility Type: ' + new_DC_Fast_df.at[i, 'Facility_Type']\
    + '<br>' + '<br>' +'Phone: ' + new_DC_Fast_df.at[i, 'Station_Phone'] + '<br>' + '<br>' +'Business Hours: ' + new_DC_Fast_df.at[i, 'Business_Hours']
    
    #add info to map and define the incon
    #https://fontawesome.com/icons?d=gallery for charging station icon
    #use with: prefix='fa'
    folium.Marker(location = [lat, lng], popup= popup, icon = folium.Icon(color='red', prefix='fa', icon='charging-station')).add_to(base_map)

#save as html file
base_map.save('DC_fast_Locations.html')
base_map




In [ ]:
#put the chloropleth and the markers on one chart via layers
#EXAMPLE CODE 

#m = ELEC_Stations_in_USA.explore(
    #column="",  # make choropleth based on "POP2010" column
   # scheme="naturalbreaks",  # use mapclassify's natural breaks scheme
    #legend=True,  # show legend
    #k=10,  # use 10 bins
    #tooltip=False,  # hide tooltip
   # popup=["POP2010", "POP2000"],  # show popup (on-click)
    #legend_kwds=dict(colorbar=False),  # do not use colorbar
    #name="chicago",  # name of the layer in the map
#)

#groceries.explore(
   # m=m,  # pass the map object
   # color="red",  # use red color on all points
   # marker_kwds=dict(radius=5, fill=True),  # make marker radius 10px with fill
   # tooltip="Address",  # show "name" column in the tooltip
   # tooltip_kwds=dict(labels=False),  # do not show column label in the tooltip
   # name="groceries",  # name of the layer in the map
#)

#folium.TileLayer("CartoDB positron", show=False).add_to(
  #  m
#)  # use folium to add alternative tiles
#folium.LayerControl().add_to(m)  # use folium to add layer control

#m  # show map